In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
stats = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')
schedule = pd.read_csv('../../data/schedule_final.csv')

In [3]:
stats.head()

,Unnamed: 0,season,week,team,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value,team_full
0,0,2014,1,ARI,C. Palmer,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals
1,1,2014,1,ATL,M. Ryan,NaN,NaN,NaN,NaN,NaN,Atlanta Falcons
2,2,2014,1,BAL,J. Flacco,NaN,NaN,NaN,NaN,NaN,Baltimore Ravens
3,3,2014,1,BUF,E. Manuel,NaN,NaN,NaN,NaN,NaN,Buffalo Bills
4,4,2014,1,CAR,D. Anderson,NaN,NaN,NaN,NaN,NaN,Carolina Panthers


In [4]:
schedule.head()

,Unnamed: 0,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,home_qb_abv,away_qb_abv
0,0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,R. Wilson,A. Rodgers
1,1,2014-09-07,2014,1,REG,Denver Broncos,Indianapolis Colts,31,24,Peyton Manning,Andrew Luck,P. Manning,A. Luck
2,2,2014-09-07,2014,1,REG,Philadelphia Eagles,Jacksonville Jaguars,34,17,Nick Foles,Chad Henne,N. Foles,C. Henne
3,3,2014-09-07,2014,1,REG,Kansas City Chiefs,Tennessee Titans,10,26,Alex Smith,Jake Locker,A. Smith,J. Locker
4,4,2014-09-07,2014,1,REG,Los Angeles Rams,Minnesota Vikings,6,34,Shaun Hill,Matt Cassel,S. Hill,M. Cassel
